# Using MPI from Jupyter at NERSC

* This notebook shows how you can use MPI in a job allocation on Cori.
* In this example the notebook itself is not running on a node in the job allocation.
* It is running "outside" the compute nodes.

## Overview

* First we'll start up an ipyparallel cluster in a job allocation on Cori compute nodes.
* When it is running, we will make a connection to the cluster controller process.
* We'll send code to worker processes (engines) to execute code in parallel.
* The example will be simple: Using a Monte Carlo technique to estimate the value of pi.

## Enable interaction with the batch queue

* We need to issue Slurm commands like `sbatch`, `squeue`, and `scancel` from our notebook
* Enable this by loading the `slurm_magic` package: Slurm magic commands for Jupyter

In [ ]:
%load_ext slurm_magic

## Start an ipyparallel cluster on some compute nodes

* To do this we'll submit a job using the `%%sbatch` cell magic
* We'll use the submitted job's ID to make contact with it, once it's up, from the notebook
* We start an `ipcontroller` process that coordinates the worker processes
* Finally we launch `ipengine` worker processes --- just Python processes waiting for input

In [ ]:
%%sbatch
#!/bin/bash
#SBATCH --constraint=haswell
#SBATCH --nodes=2
#SBATCH --partition=debug
#SBATCH --time=30

module load python/3.6-anaconda-5.2

# Get IP address of head node
head_ip=$(ip addr show ipogif0 | grep '10\.' | awk '{print $2}' | awk -F'/' '{print $1}')

# Unique cluster ID for this job
cluster_id=cori_${SLURM_JOB_ID}

# Cluster controller
ipcontroller --ip="$head_ip" --cluster-id=$cluster_id &
sleep 10

# Compute engines
srun -u -n 32 ipengine --cluster-id=$cluster_id

## Get the cluster ID from the job ID

In [ ]:
job_id = _.split()[-1]
job_id

In [ ]:
cluster_id = "cori_" + job_id
cluster_id

## Is the job running yet?

In [ ]:
%squeue -u rthomas

## Establish connection from the notebook to the compute nodes

In [ ]:
import ipyparallel as ipp
c = ipp.Client(timeout=60, cluster_id=cluster_id)
[str(i) for i in c.ids].join()

## Initialize MPI

* With `mpi4py` the import actually calls `MPI_Init()` under the hood
* The `%%px` cell magic means "execute this cell on all the workers"

In [ ]:
%%px
from mpi4py import MPI

## The Customary MPI "Hello World"

In [ ]:
%%px
print("Hello world from rank", MPI.COMM_WORLD.rank, "of" MPI.COMM_WORLD.size)

## Let's do something more interesting, estimate Pi

* Let's import a couple more packages we need
* Remember, all the workers need to do this

In [ ]:
%%px
import random
import math

## Set up a "dart board" function

* Consider a unit square with a quarter-circle centered in the lower left-hand corner
* Distribute "darts" uniformly over the unit square
* Count how many "darts" land inside the quarter circle
* The ratio of that number of "darts" to the total number "thrown" can be used to estimate pi
* The answer gets better the more "darts" are thrown

In [ ]:
%%px
def work(trials):
    count = 0.0
    for i in range(trials):
        x = random.random()
        y = random.random()
        if x ** 2 + y ** 2 < 1.0:
            count += 1.0
    return count

## Now the MPI part

* Each MPI rank initializes its own random number generator and throws the same number of "darts"
* The total number of "darts" in each rank's circle are `MPI_Gather`ed
* MPI rank 0 tallies them up and multiplies by 4 to get a unit circle's area
* Finally we output the answer and see how well we did

In [ ]:
%%px
def compute_pi(trials):
    
    # Useful MPI information
    
    mpi_comm = MPI.COMM_WORLD
    mpi_size = mpi_comm.size
    mpi_rank = mpi_comm.rank
    mpi_root = mpi_rank == 0
    
    # Initialize random number generator
    
    random.seed(192837465 + mpi_rank)
    
    # Distribute work, gather results, and report answer
    
    count = work(trials)
    count = mpi_comm.gather(count, root=0)
    if mpi_root:
        total_count = sum(count)
        total_trials = mpi_size * trials
        estimated_pi = 4.0 * total_count / total_trials
        print("Total Count:   ", total_count)
        print("Total Trials:  ", total_trials)
        print("Estimate of pi:", estimated_pi, math.pi, abs(estimated_pi - math.pi)) 

## Time to compute pi!

In [ ]:
%%px
compute_pi(10000000)

## Example of just running on one rank (not in the notebook)

In [ ]:
%%px --targets 0
work(10000000)

## Cancel the job --- our notebook stays up

In [ ]:
%scancel $job_id